In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
#TEDDIE - import base64
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
#TEDDIE - Fixed to the expected information and adjusted to repeated confusion with the animal_shelter naming
from AnimalShelterCRUD import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
#TEDDIE - fix to correct username
username = "Teodoir"
#TEDDIE - set as the correct password
password = "Pa$$word"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################

#TEDDIE - 
app = JupyterDash('CS340_TO_Dash')

#FIX ME Add in Grazioso Salvare’s logo

#TEDDIE - add image uploaded from resources
image_filename = 'GSLogo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    
    #TEDDIE - replace the html line of code to align with design needs and add clickable link
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode(), href='www.snhu.edu'))),
    
    #TEDDIE -  Include unique identifier for dash with separating lines
    html.Hr(),
    html.Center(html.B(html.P('Teodoir OCeallaighs CS-340 Dashboard'))),
    html.Hr(),

    #TEDDIE - provide instructions for using filter buttons
    html.P('Select a filter to see a specific type of potential rescue breed or all dogs:'),
       
    #TEDDIE - Create radio buttons for filtering
        dcc.RadioItems(
            id='filter-type',
            
            #TEDDIE - initialize and visualize radio buttons for the various rescue types
            options=[
                
            #TEDDIE - label Water filter button
            {'label': 'Water', 'value': 'W'},
            #TEDDIE - label Mountain or Wilderness filter button
            {'label': 'Mountain or Wilderness', 'value': 'MW'},
            #TEDDIE - label Disaster or Individual Tracking filter button
            {'label': 'Disaster or Individual Tracking', 'value': 'DIT'},
            #TEDDIE - Added filter to see all dogs in event of potential desire to view other breeds and ages
            #TEDDIE - Side note: I call dogs pupperonis
            {'label': 'View All Dogs', 'value': 'PUPPERONI'},
            #TEDDIE - label removing filters button
            {'label': 'Remove Current Filters', 'value': 'ERASE'}
            ],
            #TEDDIE - Set value for removing the filters
            value='ERASE',
            labelStyle={'display': 'inline-block'}
        
    ),
    
    html.Hr(),
    #TEDDIE - provide instructions for choosing multiple dogs
    html.P('You can select up to three of the rows below to show the animals locations on the map below!'),
    
    #TEDDIE - create a slash between the radio button filters and the table
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
        #TEDDIE - creating the interactive portion of the table for the client to show data in multiple ways on the table
       
        #TEDDIE - use multi to allow for different options when interacting
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            #TEDDIE - set up the map!
            id='map-id',
            className='col s12 m6',
            ),
        html.Div(
            #TEDDIE -set up the dash for the pie chart
            id='graph-id',
            className='col s12 m6',
            )
        ])
    ])


#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        #TEDDIE - use if/else if to ad filtering criteria for radio buttons on dash
        
        #TEDDIE -set filter for Disaster or Individual Tracking as outlined by GS
        if filter_type == 'W':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                          {'$or': [
                                                              {'breed': 'Newfoundland'},
                                                              {'breed': 'Chesa Bay Retr'},
                                                              {'breed': 'Labrador Retriever Mix'}]}, 
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]})))
                                                
        #TEDDIE -set filter for Mountain or Wilderness as outlined by GS
        elif filter_type == 'MW':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Alaskan Malamute'},
                                                              {'breed': 'Old English Sheepdog'},
                                                              {'breed': 'Rottweiler'},
                                                              {'breed': 'Siberian Husky'}]},
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]})))
            
        #TEDDIE -set filter for Disaster or Individual Tracking as outlined by GS
        elif filter_type == 'DIT':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'Doberman Pinscher'},
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Golden Retriever'},
                                                              {'breed': 'Bloodhound'},
                                                              {'breed': 'Rottweiler'}]},
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                          }]})))
        
        #TEDDIE - filter for any dog entirely
        elif filter_type == 'PUPPERONI':
            df = pd.DataFrame(list(shelter.read({"animal_type":"Dog"})))
        
        #TEDDIE - clear filter choices to return all data
        elif filter_type == 'ERASE':
            df = pd.DataFrame.from_records(shelter.read({}))

        #TEDDIE -set params for filtering
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]



@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows'),
     Input('datatable-id', 'selected_columns')])
def update_map(viewData, selected_rows, selected_columns):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.

    #TEDDIE - provide data currently being viewed for selection in relation to the map pins
    dff = pd.DataFrame.from_dict(viewData)
    
    #TEDDIE -set the initial selected rows to 0 and iterate up to three once selected
    if selected_rows == []:
        selected_rows = [0]
    
    #TEDDIE - If 1 row is selected show the map pin for the one
    if len(selected_rows) == 1:
        return [
            dl.Map(style={'width':'800px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
        
                #TEDDIE -use the previous geolocation data from Mod 6 to return data attached to pin(s)
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        html.H4("Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15])
                    ])
                ])
            ])
        ]

    
    #TEDDIE - If 2 rows are selected show the map pins for the two
    elif len(selected_rows) == 2:
        return [
            dl.Map(style={'width':'800px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            
                #TEDDIE -use the previous geolocation data from Mod 6 to return data attached to pin(s)
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        html.H4("Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15])
                    ])
                ]),
                #TEDDIE -use the previous geolocation data from Mod 6 to return data attached to pin(s)
                dl.Marker(position=[(dff.iloc[selected_rows[1],13]), (dff.iloc[selected_rows[1],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[1],4]),
                    dl.Popup([
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[1],4]),
                        html.H4("Name"),
                        html.P(dff.iloc[selected_rows[1],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[1],12]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[1],15])
                    ])
                ])
            ])
        ]

    #TEDDIE - If 3 rows are selected show the map pins for up to three
    elif len(selected_rows) == 3:
        return [
            dl.Map(style={'width':'800px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            
                #TEDDIE -use the previous geolocation data from Mod 6 to return data attached to pin(s)
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        html.H4("Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15])
                    ])
                ]),
                #TEDDIE -use the previous geolocation data from Mod 6 to return data attached to pin(s)
                dl.Marker(position=[(dff.iloc[selected_rows[1],13]), (dff.iloc[selected_rows[1],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[1],4]),
                    dl.Popup([
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[1],4]),
                        html.H4("Name"),
                        html.P(dff.iloc[selected_rows[1],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[1],12]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[1],15])
                    ])
                ]),
                #TEDDIE -use the previous geolocation data from Mod 6 to return data attached to pin(s)
                dl.Marker(position=[(dff.iloc[selected_rows[2],13]), (dff.iloc[selected_rows[2],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[2],4]),
                    dl.Popup([
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[2],4]),
                        html.H4("Name"),
                        html.P(dff.iloc[selected_rows[2],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[2],12]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[2],15])
                    ])
                ])
            ])
        ]

#TEDDIE - create the pie chart!
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    #TEDIE - import data from view
    dff = pd.DataFrame.from_dict(viewData)
    #TEDDIE - create vals for breed name and count to fill in pie chart labels and percentages
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    #TEDIE -  return visual of pie chart!
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800, 
                height=500   
            )
        )
    ]
app